In [1]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch import optim, Tensor
import torch.functional as F
import math
from tqdm.notebook import tqdm
from chords_dataset import ChordsDataset
from model_helpers import NLP, preprocess_text
from model import Transformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv('./model_data/lyrics_processed.csv')
dataset = ChordsDataset(df, NLP)
train_i, test_i, validation_i = dataset.get_train_test_valid_indexes(0.9,0.09,0.01)



In [2]:
import random
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(input_size, hidden_size)
        self.inLinear = nn.Linear(input_size,hidden_size).to(device)
        self.out = nn.Linear(hidden_size,hidden_size).to(device)
        self.gru = nn.GRU(hidden_size, hidden_size).to(device)

    def forward(self, input, hidden):
        embedded = self.inLinear(input.view(1, 1, -1))
        output = nn.functional.relu(embedded)
        output, hidden = self.gru(output, hidden)
        hidden = self.out(hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size).to(device)
        self.gru = nn.GRU(hidden_size, hidden_size).to(device)
        # self.out = nn.Linear(hidden_size, output_size)
        self.out2 = nn.Linear(hidden_size,output_size).to(device)
        self.softmax = nn.LogSoftmax(dim=2).to(device)

    def forward(self, input, hidden):
        output = self.embedding(input.to(device)).view(1, 1, -1)
        output = nn.functional.relu(output).to(device)
        output, hidden = self.gru(output, hidden)
        # output = self.out(output)
        output = self.out2(nn.functional.relu(output).to(device))
        output = self.softmax(output)
        # print("Input of decoder after softmax  + sum",output.shape, output.sum())
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [3]:
teacher_forcing_ratio = 0.5


def train(input_tensor: Tensor, target_tensor: Tensor, encoder: nn.Module, decoder: nn.Module, encoder_optimizer, decoder_optimizer, criterion, max_length=7039):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([dataset.chord2id["<SOS>"]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden  = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(
                decoder_output.view(1,-1,1).to(device),
                target_tensor[di].view(1,1).type(torch.LongTensor).to(device)
            )

            decoder_input = target_tensor[di].view(1,1).type(torch.LongTensor)    # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(
                decoder_output.view(1,-1,1).to(device),
                target_tensor[di].view(1,1).type(torch.LongTensor).to(device)
            )
            # print(loss)
            # print(decoder_output.sum(), target_tensor[di])
            if decoder_input.item() == dataset.chord2id["<EOS>"]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [4]:

import time

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def predictLyrics(lyrics : str) -> list:
    
    return ["COu", "Cou"]

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for i,index in enumerate(train_i):
        elem = dataset[index]
        source, target = elem["lyrics"].reshape(-1,1,100).to(device), elem["chords"].reshape(-1,1).to(device)
        loss = train(source, target, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if i % print_every == 1:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(timeSince(start, i / len(train_i)), (i / len(train_i)) * 100, print_loss_avg)

        if i % print_every == 1:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
        # if iter % plot_every == 0:
        #     showPlot(plot_losses)

    showPlot(plot_losses)


In [17]:
hidden_size = 256
output_dim = len(dataset.chords_set)
encoder1 = EncoderRNN(100, hidden_size).to(device)
attn_decoder1 = DecoderRNN(hidden_size, output_dim).to(device)

trainIters(encoder1, attn_decoder1, len(dataset), print_every=20, learning_rate= 5e-4)

0m 1s (- 500m 42s) 0.004714090416254184 0.7822369468248361


KeyboardInterrupt: 

In [18]:
def evaluate(encoder, decoder, sentence, max_length=7039):
    with torch.no_grad():
        input_tensor = Tensor([dataset.vectorize(w) for w in preprocess_text( sentence)]).to(device)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)


        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([dataset.chord2id["<SOS>"]], device=device)

        decoder_hidden = encoder_hidden

        decoded_words = []
        # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == dataset.chord2id["<EOS>"]:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(dataset.id2chord[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
lyrics = '''
I am not, I am feeling glad, 
I've got sunshine in a bag
I'm useless, but not for long
The future is coming on 
'''
evaluate(encoder1, attn_decoder1,lyrics, 100 )

['<SOS>',
 '<SOS>',
 '<SOS>',
 '<SOS>',
 '<SOS>',
 'G',
 'C',
 'C',
 'G',
 'C',
 'C',
 'G',
 'C',
 'C',
 'G',
 'C',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'G']